In [71]:
import uuid

from langchain_core.runnables.config import RunnableConfig
from langgraph.checkpoint.memory import MemorySaver
from langgraph.func import entrypoint, task
from langgraph.types import Command, interrupt
from langchain_core.tools import tool
from loguru import logger
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
@tool
def write_essay(topic: str) -> str:
    """Write an essay about the given topic."""
    return f"An essay about topic: {topic}"

In [87]:
write_essay.model_dump()

{'name': 'write_essay',
 'description': 'Write an essay about the given topic.',
 'args_schema': langchain_core.utils.pydantic.write_essay,
 'return_direct': False,
 'verbose': False,
 'tags': None,
 'metadata': None,
 'handle_tool_error': False,
 'handle_validation_error': False,
 'response_format': 'content',
 'func': <function __main__.write_essay(topic: str) -> str>,
 'coroutine': None}

In [72]:
@tool
async def write_essay(topic: str) -> str:
    """Write an essay about the given topic."""
    return f"An essay about topic: {topic}"
    # agent = Agent(model="google-gla:gemini-1.5-flash", name="essay_writer")
    # return (await agent.run(f"Write an essay about {topic}")).data


@entrypoint(checkpointer=MemorySaver())
async def workflow(topic: str) -> dict:
    """A simple workflow that writes an essay and asks for a review."""
    essay = await write_essay(topic=topic)
    # is_approved = False
    is_approved = interrupt(
        {
            # Any json-serializable payload provided to interrupt as argument.
            # It will be surfaced on the client side as an Interrupt when streaming data
            # from the workflow.
            "essay": essay,  # The essay we want reviewed.
            # We can add any additional information that we need.
            # For example, introduce a key called "action" with some instructions.
            "action": "Please approve/reject the essay",
        }
    )

    return {
        "essay": essay,  # The essay that was generated
        "is_approved": is_approved,  # Response from HIL
    }

In [69]:
thread_id = str(uuid.uuid4())

config = RunnableConfig(configurable={"thread_id": thread_id})


In [70]:
res = await workflow.ainvoke("cat", config=config)
res

In [ ]:
res.

In [63]:
res = await workflow.ainvoke(Command(resume=False), config=config)
res

{'essay': 'An essay about topic: cat', 'is_approved': False}

In [13]:
thread_id = str(uuid.uuid4())

config = {"configurable": {"thread_id": thread_id}}

async for item in workflow.astream("cat", config=config):
    print(item)

/tmp/ipykernel_54703/1648118602.py:5: LogfireNotConfiguredWarning: No logs or spans will be created until `logfire.configure()` has been called. Set the environment variable LOGFIRE_IGNORE_NO_CONFIG=1 or add ignore_no_config=true in pyproject.toml to suppress this warning.
  return (await agent.run(f"Write an essay about {topic}")).data


{'write_essay': "## The Enigmatic Cat: A Study in Independence and Affection\n\nThe domestic cat, *Felis catus*, is a creature of paradox. Simultaneously aloof and affectionate, independent yet deeply connected to its human companions, the cat defies simple categorization.  Its enigmatic nature, coupled with its undeniable charm, has captivated human hearts for millennia, resulting in a complex and enduring relationship that continues to evolve.\n\nOne of the most striking aspects of the cat's character is its inherent independence. Unlike dogs, whose domestication is deeply rooted in a collaborative partnership, cats retain a significant portion of their wild ancestry.  This translates into a self-sufficient creature, capable of self-grooming, hunting (even in a domestic setting), and navigating its environment with remarkable agility and resourcefulness.  Their seeming indifference to human commands, often interpreted as stubbornness, is simply a reflection of this independent spirit

In [14]:
human_review = True

async for item in workflow.astream(Command(resume=human_review), config=config):
    print(item)


{'workflow': {'essay': "## The Enigmatic Cat: A Study in Independence and Affection\n\nThe domestic cat, *Felis catus*, is a creature of paradox. Simultaneously aloof and affectionate, independent yet deeply connected to its human companions, the cat defies simple categorization.  Its enigmatic nature, coupled with its undeniable charm, has captivated human hearts for millennia, resulting in a complex and enduring relationship that continues to evolve.\n\nOne of the most striking aspects of the cat's character is its inherent independence. Unlike dogs, whose domestication is deeply rooted in a collaborative partnership, cats retain a significant portion of their wild ancestry.  This translates into a self-sufficient creature, capable of self-grooming, hunting (even in a domestic setting), and navigating its environment with remarkable agility and resourcefulness.  Their seeming indifference to human commands, often interpreted as stubbornness, is simply a reflection of this independent

In [2]:
class Critique(BaseModel):
    funny: bool
    reason: str = ""
    pointers: list[str] = Field(default_factory=list)


agent = Agent(model="google-gla:gemini-1.5-flash", system_prompt="Use the tool to tell jokes", name="joker_agent")

joker = Agent(model="google-gla:gemini-1.5-flash", system_prompt="Tell knock knock jokes", name="joker_tool")

critic = Agent(
    model="google-gla:gemini-1.5-flash",
    system_prompt="Critique the joke as funny or not funny. If not funny, give a reason for your opinion and pointers for improvement",
    result_type=Critique,
    name="joke_critic",
)


@agent.tool_plain
async def joke_teller(premise: str) -> str:
    "Tool to tell jokes about anything"
    return (await joker.run(premise)).data


@agent.result_validator
async def validate_joke(ctx: RunContext, joke: str) -> str:
    critique = (
        await critic.run(
            user_prompt="Critique the joke as funny or not funny. If not funny, give a reason for your opinion and pointers for improvement. It will always be a knock knock joke so don't mention that",
            message_history=ctx.messages,
        )
    ).data
    logger.info(f"Joke Critique: {critique.model_dump_json()}")
    return joke

In [3]:
joke = await agent.run("Tell me a joke about the justice league")

14:27:11.694 joker_agent run prompt=Tell me a joke about the justice league
14:27:11.694   preparing model and tools run_step=1
14:27:11.695   model request


Logfire project URL: ]8;id=556578;https://logfire.pydantic.dev/HamzaFarhan/knd\https://logfire.pydantic.dev/HamzaFarhan/knd]8;;\

14:27:14.634   handle model response
14:27:14.635     running tools=['joke_teller']
14:27:14.635     joker_tool run prompt=Justice League
14:27:14.636       preparing model and tools run_step=1
14:27:14.636       model request
14:27:15.392       handle model response
14:27:15.399   preparing model and tools run_step=2
14:27:15.400   model request
14:27:16.174   handle model response
14:27:16.176     joke_critic run prompt=Critique the joke as funny or not funny. If not funny, give a ...nt. It will always be a knock knock joke so don't mention that
14:27:16.178       preparing model and tools run_step=1
14:27:16.180       model request
14:27:17.858       handle model response
14:27:17.873     not funny [unfunny_joke]


In [4]:
joke.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='Use the tool to tell jokes', dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='Tell me a joke about the justice league', timestamp=datetime.datetime(2025, 1, 25, 14, 27, 11, 694489, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='joke_teller', args=ArgsDict(args_dict={'premise': 'Justice League'}), tool_call_id=None, part_kind='tool-call')], model_name='gemini-1.5-flash', timestamp=datetime.datetime(2025, 1, 25, 14, 27, 14, 633532, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='joke_teller', content="Knock knock.\n\nWho's there?\n\nJustice.\n\nJustice who?\n\nJustice League of extraordinary jokes!  (or... Justice League of heroes!)\n", tool_call_id=None, timestamp=datetime.datetime(2025, 1, 25, 14, 27, 15, 397216, tzinfo=datetime.timezone.utc), part_kind='tool-return')], kind='request'),
 ModelRespon

In [5]:
print(joke.data)

Knock knock.

Who's there?

Justice.

Justice who?

Justice League of extraordinary jokes!  (or... Justice League of heroes!)

